<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Valid_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Best Model - ViT Fused Validation
# Notebook to Evaluate ViT on 100 and __% datasets residing in a single directory.

In [ ]:
import torch
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm
from torch.utils.data import DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"


In [3]:
import os

test_image_file_paths = []
test_image_genre_labels = []
test_enc_file_paths = []
test_enc_genre_labels = []

test_labels_file = "/content/Dataset(s)/joint-food-101/test_dataset.txt"

with open(test_labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split(', ')
        filename = parts[0].strip()
        labels = parts[1].strip()

        if filename.endswith("_3.png"):
            test_enc_file_paths.append(filename)
            test_enc_genre_labels.append(labels)

        elif filename.endswith("_4.png"):
            test_image_file_paths.append(filename)
            test_image_genre_labels.append(labels)

In [4]:
len(test_image_file_paths) , len(test_enc_file_paths)

(22716, 22716)

In [5]:
unique_labels = set(test_enc_genre_labels)
unique_labels_list = list(unique_labels)

print("Unique Labels:", unique_labels_list)

Unique Labels: ['chicken_quesadilla', 'fish_and_chips', 'onion_rings', 'pork_chop', 'bibimbap', 'sushi', 'donuts', 'huevos_rancheros', 'spaghetti_carbonara', 'fried_calamari', 'peking_duck', 'foie_gras', 'french_fries', 'prime_rib', 'pho', 'cheesecake', 'macarons', 'pad_thai', 'grilled_cheese_sandwich', 'takoyaki', 'paella', 'lobster_bisque', 'escargots', 'caesar_salad', 'ice_cream', 'spring_rolls', 'deviled_eggs', 'cup_cakes', 'panna_cotta', 'steak', 'breakfast_burrito', 'pizza', 'strawberry_shortcake', 'baklava', 'carrot_cake', 'pancakes', 'macaroni_and_cheese', 'garlic_bread', 'dumplings', 'sashimi', 'croque_madame', 'guacamole', 'frozen_yogurt', 'tacos', 'churros', 'gnocchi', 'shrimp_and_grits', 'crab_cakes', 'hummus', 'cannoli', 'ramen', 'ceviche', 'baby_back_ribs', 'gyoza', 'cheese_plate', 'chicken_wings', 'bruschetta', 'creme_brulee', 'tiramisu', 'caprese_salad', 'mussels', 'apple_pie', 'beignets', 'eggs_benedict', 'clam_chowder', 'filet_mignon', 'omelette', 'chicken_curry', 're

In [6]:
len(unique_labels_list)

101

In [7]:
import random

percentage = 0.3

label_counts = {label: 0 for label in unique_labels_list}

test_num_samples_to_include_per_label = {}
for label in unique_labels_list:
    label_count = test_enc_genre_labels.count(label)
    num_samples_to_include = int(label_count * percentage)
    test_num_samples_to_include_per_label[label] = num_samples_to_include


test_filtered_enc_file_paths = []
test_filtered_enc_genre_labels = []


test_selected_samples_per_label = {label: 0 for label in unique_labels_list}
for path, label in zip(test_enc_file_paths, test_enc_genre_labels):
    if test_selected_samples_per_label[label] < test_num_samples_to_include_per_label[label]:
        test_filtered_enc_file_paths.append(path)
        test_filtered_enc_genre_labels.append(label)
        test_selected_samples_per_label[label] += 1

In [8]:
len(test_filtered_enc_file_paths) , len(test_filtered_enc_genre_labels)

(6770, 6770)

In [9]:
# Create new lists to combine data from both lists
test_file_paths = test_image_file_paths + test_filtered_enc_file_paths
test_genre_labels = test_image_genre_labels + test_filtered_enc_genre_labels

In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder1 = LabelEncoder()
transformed_test_genre_labels = label_encoder1.fit_transform(test_genre_labels)

In [11]:
label_to_id_mapping = {label: id for label, id in zip(label_encoder1.classes_, label_encoder1.transform(label_encoder1.classes_))}
id_to_label_mapping = {id: label for id, label in enumerate(label_encoder1.classes_)}

In [12]:
# Convert int64 to int in id_to_label_mapping and label_to_id_mapping
id_to_label_mapping = {int(id): label for id, label in id_to_label_mapping.items()}
label_to_id_mapping = {label: int(id) for label, id in label_to_id_mapping.items()}

In [13]:
id_to_label_mapping

{0: 'apple_pie',
 1: 'baby_back_ribs',
 2: 'baklava',
 3: 'beef_carpaccio',
 4: 'beef_tartare',
 5: 'beet_salad',
 6: 'beignets',
 7: 'bibimbap',
 8: 'bread_pudding',
 9: 'breakfast_burrito',
 10: 'bruschetta',
 11: 'caesar_salad',
 12: 'cannoli',
 13: 'caprese_salad',
 14: 'carrot_cake',
 15: 'ceviche',
 16: 'cheese_plate',
 17: 'cheesecake',
 18: 'chicken_curry',
 19: 'chicken_quesadilla',
 20: 'chicken_wings',
 21: 'chocolate_cake',
 22: 'chocolate_mousse',
 23: 'churros',
 24: 'clam_chowder',
 25: 'club_sandwich',
 26: 'crab_cakes',
 27: 'creme_brulee',
 28: 'croque_madame',
 29: 'cup_cakes',
 30: 'deviled_eggs',
 31: 'donuts',
 32: 'dumplings',
 33: 'edamame',
 34: 'eggs_benedict',
 35: 'escargots',
 36: 'falafel',
 37: 'filet_mignon',
 38: 'fish_and_chips',
 39: 'foie_gras',
 40: 'french_fries',
 41: 'french_onion_soup',
 42: 'french_toast',
 43: 'fried_calamari',
 44: 'fried_rice',
 45: 'frozen_yogurt',
 46: 'garlic_bread',
 47: 'gnocchi',
 48: 'greek_salad',
 49: 'grilled_chees

In [14]:
from datasets import Dataset

val_data = {'image': test_file_paths, 'label': transformed_test_genre_labels}
ds_val = Dataset.from_dict(val_data)

In [15]:
import PIL.Image as pil

def transform(examples):
  inputs = image_processor([pil.open(img).convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]
  return inputs

In [16]:
val_dataset = ds_val.with_transform(transform)

In [17]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [18]:
batch_size = 32
valid_dataset_loader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [19]:
checkpoint = 9000

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

In [21]:
while checkpoint <= 54000:
    model = ViTForImageClassification.from_pretrained(f"/content/Model/Models-Train-28/checkpoint-{checkpoint}").to(device)
    image_processor = ViTImageProcessor.from_pretrained(f"/content/Model/Models-Train-28/checkpoint-{checkpoint}")
    model = model.to(device)
    model.eval()
    print()
    print("Evaluation at checkpoint: ", checkpoint)
    print()

    predictions, labels = [], []

    for batch in valid_dataset_loader:
        pixel_values = batch["pixel_values"].to(device)
        label_ids = batch["labels"].to(device)

        outputs = model(pixel_values=pixel_values, labels=label_ids)
        logits = outputs.logits.detach().cpu()

        predictions.extend(logits.argmax(dim=-1).tolist())
        labels.extend(label_ids.tolist())

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    print("Accuracy: ", accuracy)
    print("precision: ", precision)
    print("f1_score: ", f1)
    print("recall", recall)
    print(classification_report(labels, predictions))
    checkpoint = checkpoint + 9000

loading configuration file /content/Model/Models-Train-28/checkpoint-9000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "apple_pie",
    "1": "baby_back_ribs",
    "2": "baklava",
    "3": "beef_carpaccio",
    "4": "beef_tartare",
    "5": "beet_salad",
    "6": "beignets",
    "7": "bibimbap",
    "8": "bread_pudding",
    "9": "breakfast_burrito",
    "10": "bruschetta",
    "11": "caesar_salad",
    "12": "cannoli",
    "13": "caprese_salad",
    "14": "carrot_cake",
    "15": "ceviche",
    "16": "cheese_plate",
    "17": "cheesecake",
    "18": "chicken_curry",
    "19": "chicken_quesadilla",
    "20": "chicken_wings",
    "21": "chocolate_cake",
    "22": "chocolate_mousse",
    "23": "churros",
    "24": "clam_chowder",


Evaluation at checkpoint:  9000



loading configuration file /content/Model/Models-Train-28/checkpoint-18000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "apple_pie",
    "1": "baby_back_ribs",
    "2": "baklava",
    "3": "beef_carpaccio",
    "4": "beef_tartare",
    "5": "beet_salad",
    "6": "beignets",
    "7": "bibimbap",
    "8": "bread_pudding",
    "9": "breakfast_burrito",
    "10": "bruschetta",
    "11": "caesar_salad",
    "12": "cannoli",
    "13": "caprese_salad",
    "14": "carrot_cake",
    "15": "ceviche",
    "16": "cheese_plate",
    "17": "cheesecake",
    "18": "chicken_curry",
    "19": "chicken_quesadilla",
    "20": "chicken_wings",
    "21": "chocolate_cake",
    "22": "chocolate_mousse",
    "23": "churros",
    "24": "clam_chowder"

Accuracy:  0.7401139523841823
precision:  0.7568221878990345
f1_score:  0.7452740496727903
recall 0.7401139523841823
              precision    recall  f1-score   support

           0       0.72      0.81      0.76       304
           1       0.83      0.82      0.82       287
           2       0.91      0.82      0.86       293
           3       0.71      0.68      0.69       288
           4       0.44      0.53      0.48       292
           5       0.90      0.79      0.84       291
           6       0.72      0.74      0.73       291
           7       0.74      0.74      0.74       292
           8       0.75      0.74      0.75       293
           9       0.80      0.70      0.75       278
          10       0.66      0.84      0.74       300
          11       0.76      0.84      0.80       295
          12       0.75      0.78      0.76       299
          13       0.83      0.83      0.83       286
          14       0.77      0.73      0.75       300
          15      

All model checkpoint weights were used when initializing ViTForImageClassification.

All the weights of ViTForImageClassification were initialized from the model checkpoint at /content/Model/Models-Train-28/checkpoint-18000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ViTForImageClassification for predictions without further training.
loading configuration file /content/Model/Models-Train-28/checkpoint-18000/preprocessor_config.json
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}



...


Evaluation at checkpoint:  45000



loading configuration file /content/Model/Models-Train-28/checkpoint-54000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "apple_pie",
    "1": "baby_back_ribs",
    "2": "baklava",
    "3": "beef_carpaccio",
    "4": "beef_tartare",
    "5": "beet_salad",
    "6": "beignets",
    "7": "bibimbap",
    "8": "bread_pudding",
    "9": "breakfast_burrito",
    "10": "bruschetta",
    "11": "caesar_salad",
    "12": "cannoli",
    "13": "caprese_salad",
    "14": "carrot_cake",
    "15": "ceviche",
    "16": "cheese_plate",
    "17": "cheesecake",
    "18": "chicken_curry",
    "19": "chicken_quesadilla",
    "20": "chicken_wings",
    "21": "chocolate_cake",
    "22": "chocolate_mousse",
    "23": "churros",
    "24": "clam_chowder"

Accuracy:  0.7512039612019263
precision:  0.7615822969076002
f1_score:  0.753444621153516
recall 0.7512039612019263
              precision    recall  f1-score   support

           0       0.78      0.76      0.77       304
           1       0.89      0.80      0.84       287
           2       0.40      0.87      0.55       293
           3       0.53      0.73      0.61       288
           4       0.59      0.49      0.54       292
           5       0.82      0.79      0.81       291
           6       0.74      0.72      0.73       291
           7       0.84      0.73      0.78       292
           8       0.74      0.74      0.74       293
           9       0.75      0.72      0.73       278
          10       0.72      0.79      0.75       300
          11       0.89      0.81      0.85       295
          12       0.76      0.78      0.77       299
          13       0.80      0.84      0.82       286
          14       0.82      0.71      0.76       300
          15       

All model checkpoint weights were used when initializing ViTForImageClassification.

All the weights of ViTForImageClassification were initialized from the model checkpoint at /content/Model/Models-Train-28/checkpoint-54000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ViTForImageClassification for predictions without further training.
loading configuration file /content/Model/Models-Train-28/checkpoint-54000/preprocessor_config.json
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}




Evaluation at checkpoint:  54000

Accuracy:  0.7542901716068643
precision:  0.7641506233701264
f1_score:  0.7567883067711431
recall 0.7542901716068643
              precision    recall  f1-score   support

           0       0.71      0.79      0.75       304
           1       0.88      0.79      0.83       287
           2       0.87      0.82      0.84       293
           3       0.63      0.71      0.67       288
           4       0.56      0.51      0.53       292
           5       0.81      0.80      0.80       291
           6       0.76      0.73      0.74       291
           7       0.81      0.74      0.77       292
           8       0.71      0.77      0.74       293
           9       0.76      0.72      0.74       278
          10       0.71      0.82      0.76       300
          11       0.86      0.82      0.84       295
          12       0.78      0.79      0.78       299
          13       0.85      0.84      0.84       286
          14       0.80      0.73    